# Carga, Exploración, Limpieza y Visualización Básica de un Dataset

## Exploración y primeras conclusiones 

In [4]:
 #En primer lugar, importamos las librerías necesarias y cargamos el dataset de estudio.
           
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("../Data/netflix_titles.csv")

#Primera visualización del dataset

df.head()

# df.shape --> (8807, 12)  --> 8807 filas y 12 columnas

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [5]:
# df.duplicated().sum()  --> 0  --> No existen filas duplicadas en el dataset

df.info()

# Tras ver la información básica del dataset, observamos en primer lugar que la mayoría de los datos son de tipo object.
# Vemos que existen valores nulos, cuyo tratamiento se determinará más adelante.
# También se aprecia que la columna "date_added" es de tipo object, cuando debería ser datetime, algo que habrá que corregir.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [6]:
df.columns

# Los nombres de las columnas están sin espacios y en minúsculas, es decir, en un formato que facilita su manejo y nos interesa.


Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

Una vez introducido nuestro dataset, veamos si alguna columna contiene valores que no tengan sentido.
Cabe destacar que los valores nulos no entran dentro de esta categoría y su estudio se realiza más adelante.
Con esto lograremos:

- Verificar si los valores son coherentes con el significado de la variable

- Identificar posibles inconsistencias o errores

- Decidir qué columna necesita limpieza y cuál no

Columna del identificador
<span style="background-color: #f0f0f02d; padding: 2px 4px; border-radius: 3px;">
'show_id'
</span>

In [7]:
len(df['show_id'].unique()) == df.shape[0]  # --> True  --> La columna 'show_id' es un identificador único para cada fila

True

Columna que indica el tipo de elemento de la tabla: Película o TV Show.

In [8]:
df['type'].unique() # --> Hay dos tipos de contenido: 'Movie' y 'TV Show', en principio no hay que hacer nada con esta columna

array(['Movie', 'TV Show'], dtype=object)

Columna <span style="background-color: #f0f0f02d; padding: 2px 4px; border-radius: 3px;">
'title'
</span>

In [9]:
df['title'].duplicated().sum() # --> Puede que no hubiese columnas duplicadas, pero sí títulos repetidos (quizás de películas distintas con el mismo nombre). En nuestro caso, no.

0

Para las columnas <span style="background-color: #f0f0f02d; padding: 2px 4px; border-radius: 3px;">
'director'
</span>, <span style="background-color: #f0f0f02d; padding: 2px 4px; border-radius: 3px;">
'cast'
</span>y
<span style="background-color: #f0f0f02d; padding: 2px 4px; border-radius: 3px;">
'country'
</span>no hay valores imposibles (más allá de un director, reparto o país inexistentes, casos que obviaremos).

In [10]:
df['director'].unique()[:10]  # --> Hay muchos valores nulos en esta columna (2634 = df['director'].isna().sum()), 
# habrá que tratarlos. 


#Además, hay directores con varios trabajos en el dataset. Algunos ejemplos de directores con varios trabajos:
df['director'].value_counts().head(10)

director
Rajiv Chilaka             19
Raúl Campos, Jan Suter    18
Marcus Raboy              16
Suhas Kadav               16
Jay Karas                 14
Cathy Garcia-Molina       13
Martin Scorsese           12
Youssef Chahine           12
Jay Chapman               12
Steven Spielberg          11
Name: count, dtype: int64

In [11]:
df['cast'].isnull().sum() # --> 825 valores nulos en la columna 'cast', habrá que tratarlos.

825

Algunas preguntas que nos podemos hacer son: ¿existen películas (o series) con exactamente el mismo cast? ¿Cuál es el actor que aparece en más castings?
Quiero ver los actores que salen n veces, tanto en series como en películas.

Obs : Esta serie de preguntas se responde brevemente en esta sección, pero podrían ser estudiadas con más detalle mediante gráficos en próximas secciones.

In [12]:
df['cast'].duplicated().any()

# Quiere cedir que no existen casting exactamente iguales en el dataset

True

In [13]:
df['cast'].value_counts()[lambda x: x > 1]

# Como value_counts() ordena de mayor a menor, los actores que salen más veces en el dataset aparecen primero.
# Por lo tanto, para ver el que aparece más veces, bastaría con consultar el primero.

# Si quisiéramos ver la lista de actores que aparecen n veces, cambiaríamos el 1 por n o, alternativamente, 
# podríamos definir la siguiente función:
def actores_con_n_apariciones(n):
    return df['cast'].value_counts()[lambda x: x > n]


¿Y si quiero filtrar todo esto por series o películas?

In [14]:
df_filtrado = df[df['type'] == 'Movie']

df_filtrado['cast'].value_counts()[lambda x: x > 1]

# Genéricamente :

def actores_con_n_apariciones_filtrado(n, tipo):
    df_filtrado = df[df['type'] == tipo]
    return df_filtrado['cast'].value_counts()[lambda x: x > n]

# Tras este prqueño análisis pasamos a la columna contry.

In [15]:
 df['country'].value_counts() # --> Una gran cantidad de películas estadounidenses, interesante para quizá algún gráfico futuro.
# Podríamos hacer otro tanto de preguntas también relacionadas con los países, pero de momento seguimos adelante.

country
United States                             2818
India                                      972
United Kingdom                             419
Japan                                      245
South Korea                                199
                                          ... 
Romania, Bulgaria, Hungary                   1
Uruguay, Guatemala                           1
France, Senegal, Belgium                     1
Mexico, United States, Spain, Colombia       1
United Arab Emirates, Jordan                 1
Name: count, Length: 748, dtype: int64

Columna <span style="background-color: #f0f0f02d; padding: 2px 4px; border-radius: 3px;">
'data_added'
</span>

In [16]:
df['date_added'].dtype  # --> object, por lo que habrá que convertirla a datetime.

# Por otro lado, la posibilidad de que existan fechas incoherentes (ie, que el día sea 35) se tratará junto con el cambio de tipo.
df['date_added'].unique()[:10]

array(['September 25, 2021', 'September 24, 2021', 'September 23, 2021',
       'September 22, 2021', 'September 21, 2021', 'September 20, 2021',
       'September 19, 2021', 'September 17, 2021', 'September 16, 2021',
       'September 15, 2021'], dtype=object)

Columan año publicado (Serie o TV-show)

In [17]:
df['release_year'].unique() # --> Los años van desde 1925 hasta 2021, no hay años futuros, lo cual es correcto. Habrá que ver si hay años con pocos datos.

array([2020, 2021, 1993, 2018, 1996, 1998, 1997, 2010, 2013, 2017, 1975,
       1978, 1983, 1987, 2012, 2001, 2014, 2002, 2003, 2004, 2011, 2008,
       2009, 2007, 2005, 2006, 1994, 2015, 2019, 2016, 1982, 1989, 1990,
       1991, 1999, 1986, 1992, 1984, 1980, 1961, 2000, 1995, 1985, 1976,
       1959, 1988, 1981, 1972, 1964, 1945, 1954, 1979, 1958, 1956, 1963,
       1970, 1973, 1925, 1974, 1960, 1966, 1971, 1962, 1969, 1977, 1967,
       1968, 1965, 1946, 1942, 1955, 1944, 1947, 1943], dtype=int64)

Columan <span style="background-color: #f0f0f02d; padding: 2px 4px; border-radius: 3px;">
'rating'
</span> (valoracion)

In [18]:
df['rating'].unique() # --> Dentro de los ratings aparecen algunos valores como "74 min" o "84 min", 
# que no son ratings; probablemente sean la duración de la película, por lo que habrá que tratarlos.

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

Columna <span style="background-color: #f0f0f02d; padding: 2px 4px; border-radius: 3px;">
'listed_in'
</span>

In [19]:
df['listed_in'].value_counts()  # --> Hay muchos géneros distintos; habrá que ver si se pueden agrupar algunos. En principio, se da por hecho que los valores son correctos.

listed_in
Dramas, International Movies                          362
Documentaries                                         359
Stand-Up Comedy                                       334
Comedies, Dramas, International Movies                274
Dramas, Independent Movies, International Movies      252
                                                     ... 
Kids' TV, TV Action & Adventure, TV Dramas              1
TV Comedies, TV Dramas, TV Horror                       1
Children & Family Movies, Comedies, LGBTQ Movies        1
Kids' TV, Spanish-Language TV Shows, Teen TV Shows      1
Cult Movies, Dramas, Thrillers                          1
Name: count, Length: 514, dtype: int64

Finalmente la columna description es muy variada y única, y el contenido no tiene restricción alguna

## Conclusiones del análisis preliminar

Tras revisar los valores de cada columna del dataset, se puede concluir que los datos son, en general, coherentes con la estructura de un catálogo audiovisual, aunque presentan varias inconsistencias importantes que deberán ser corregidas en la fase de limpieza.

Variables como show_id, type, title, release_year y listed_in muestran valores razonables y acordes a su significado. No obstante, habrá que hacer un especial énfasis en:

- Los valores nulos en director, cast y country, junto con una posible infinidad de distintas preguntas que no podemos hacer.

- La columna date_added está almacenada como texto y deberá convertirse a formato fecha.

- La columna duration mezcla dos conceptos distintos —minutos y número de temporadas—, por lo que será necesario separarla o normalizarla.

Se han detectado valores incorrectos en la columna rating, como “64 min”.

Algunas columnas de texto libre, como listed_in o description, tienen estructuras complejas (varios géneros, descripciones largas) y habrá que ver si podemos agruparlas o decidir qué hacer con estas categorías en caso de que sean importantes para el estudio.